In [1]:
# =====================================================================
# BaristaBot – FULL, RUNNABLE NOTEBOOK (FIXED)
# =====================================================================
# 1️⃣  Install and import all packages
# ---------------------------------------------------------------------
%pip install -qU langgraph==0.2.45 langchain-google-genai==2.0.4

import os
import getpass

# Fixed API key handling
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = getpass.getpass("Paste your Google API key: ").strip()
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

from typing import Annotated, Literal
from typing_extensions import TypedDict
from collections.abc import Iterable
from random import randint

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.tools import tool

# 2️⃣  Model
# ---------------------------------------------------------------------
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

# 3️⃣  State definition
# ---------------------------------------------------------------------
class OrderState(TypedDict):
    messages: Annotated[list, add_messages]  # conversation
    order: list[str]                         # structured order
    finished: bool                           # conversation end flag

# 4️⃣  System prompt
# ---------------------------------------------------------------------
BARISTABOT_SYSINT = (
    "system",
    "You are BaristaBot, an interactive café ordering system. "
    "Only discuss menu items. Help the customer build an order, confirm it, "
    "and finally call place_order. Use only the tools provided. "
    "Once place_order returns, thank the customer and say goodbye."
)
WELCOME_MSG = "Welcome to the BaristaBot café! Type `q` to quit. How may I serve you?"

# 5️⃣  Stateless tool: live menu
# ---------------------------------------------------------------------
@tool
def get_menu() -> str:
    """Return the current menu with all drinks and modifiers."""
    return """
MENU:
Coffee Drinks:
Espresso, Americano, Cold Brew

Coffee Drinks with Milk:
Latte, Cappuccino, Cortado, Macchiato, Mocha, Flat White

Tea Drinks:
English Breakfast Tea, Green Tea, Earl Grey

Tea Drinks with Milk:
Chai Latte, Matcha Latte, London Fog

Other Drinks:
Steamer, Hot Chocolate

Modifiers:
Milk: Whole (default), 2%, Oat, Almond, 2% Lactose-Free
Shots: Single, Double (default), Triple, Quadruple
Caffeine: Regular (default), Decaf
Temperature: Hot (default), Iced
Sweeteners: vanilla, hazelnut, caramel, chocolate, sugar-free vanilla
Special: extra hot, extra foam, one-pump, half-caff, etc.
Note: Soy milk out of stock today.
"""

# 6️⃣  Ordering tools (schema only – real work in order_node)
# ---------------------------------------------------------------------
@tool
def add_to_order(drink: str, modifiers: Iterable[str]) -> str:
    """Add a drink with modifiers to the order."""
    pass

@tool
def confirm_order() -> str:
    """Confirm the current order with the customer."""
    pass

@tool
def get_order() -> str:
    """Get the current order."""
    pass

@tool
def clear_order() -> str:
    """Clear the current order."""
    pass

@tool
def place_order() -> int:
    """Place the final order and return estimated wait time."""
    pass

# 7️⃣  Nodes
# ---------------------------------------------------------------------
def human_node(state: OrderState) -> OrderState:
    """Prompt user and handle quit."""
    last = state["messages"][-1]
    print("BaristaBot:", last.content)
    user = input("You: ").strip()
    if user.lower() in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True
    return state | {"messages": [("user", user)]}

def chatbot_node(state: OrderState) -> OrderState:
    """LLM node with ALL tools bound."""
    if not state["messages"]:
        return state | {"messages": [AIMessage(content=WELCOME_MSG)]}
    response = llm_with_tools.invoke([BARISTABOT_SYSINT] + state["messages"])
    return state | {"messages": [response]}

def order_node(state: OrderState) -> OrderState:
    """Stateful order manipulation."""
    last = state["messages"][-1]
    order = state.get("order", [])
    placed = False
    out_msgs = []

    for tc in last.tool_calls:
        name, args = tc["name"], tc["args"]

        if name == "add_to_order":
            mods = args.get("modifiers", [])
            mod_str = ", ".join(mods) if mods else "no modifiers"
            order.append(f"{args['drink']} ({mod_str})")
            resp = f"Added to order: {args['drink']} ({mod_str})\n\nCurrent order:\n" + "\n".join(f"- {item}" for item in order)

        elif name == "confirm_order":
            print("\n" + "="*50)
            print("YOUR ORDER:")
            print("="*50)
            if order:
                for i, drink in enumerate(order, 1):
                    print(f"{i}. {drink}")
            else:
                print("(no items in order)")
            print("="*50)
            resp = input("Is this order correct? (yes/no): ").strip()

        elif name == "get_order":
            if order:
                resp = "Current order:\n" + "\n".join(f"- {item}" for item in order)
            else:
                resp = "Your order is currently empty."

        elif name == "clear_order":
            order.clear()
            resp = "Order cleared. Starting fresh!"

        elif name == "place_order":
            if not order:
                resp = "Cannot place an empty order. Please add some items first."
            else:
                print("\n" + "🎉" * 20)
                print("📦 SENDING ORDER TO KITCHEN!")
                print("🎉" * 20)
                for i, drink in enumerate(order, 1):
                    print(f"{i}. {drink}")
                print("🎉" * 20)
                eta = randint(3, 8)
                print(f"⏰ Estimated wait time: {eta} minutes")
                placed = True
                resp = f"Order placed successfully! Your order will be ready in approximately {eta} minutes."

        else:
            raise NotImplementedError(f"Unknown tool: {name}")

        out_msgs.append(
            ToolMessage(content=resp, name=name, tool_call_id=tc["id"])
        )

    return {"messages": out_msgs, "order": order, "finished": placed}

# 8️⃣  Router logic
# ---------------------------------------------------------------------
auto_tools = [get_menu]
order_tools = [add_to_order, confirm_order, get_order, clear_order, place_order]

tool_node_auto = ToolNode(auto_tools)
llm_with_tools = llm.bind_tools(auto_tools + order_tools)

def router(state: OrderState) -> Literal["tools", "ordering", "human", "__end__"]:
    if state.get("finished"):
        return END
    msg = state["messages"][-1]
    if not getattr(msg, "tool_calls", None):
        return "human"
    # Split between auto vs ordering tools
    for tc in msg.tool_calls:
        if tc["name"] in {t.name for t in auto_tools}:
            return "tools"
    return "ordering"

# 9️⃣  Build the graph
# ---------------------------------------------------------------------
graph = StateGraph(OrderState)

graph.add_node("chatbot", chatbot_node)
graph.add_node("human", human_node)
graph.add_node("tools", tool_node_auto)
graph.add_node("ordering", order_node)

graph.add_edge(START, "chatbot")
graph.add_conditional_edges("chatbot", router)
graph.add_edge("tools", "chatbot")
graph.add_edge("ordering", "chatbot")
graph.add_conditional_edges("human", lambda s: END if s.get("finished") else "chatbot")

app = graph.compile()

# 🔟  Run the café!
# ---------------------------------------------------------------------
def run_baristabot():
    """Run the BaristaBot café ordering system."""
    print("🤖 Starting BaristaBot...")
    print("=" * 60)

    try:
        config = {"recursion_limit": 100}
        final_state = app.invoke({"messages": [], "order": [], "finished": False}, config)
        print("\n" + "=" * 60)
        print("☕ Thanks for visiting BaristaBot café! Have a great day!")
        print("=" * 60)
    except KeyboardInterrupt:
        print("\n\n👋 Session interrupted. Thanks for visiting!")
    except Exception as e:
        print(f"\n❌ An error occurred: {e}")
        print("Please check your API key and try again.")

if __name__ == "__main__":
    run_baristabot()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 8.4 MB/s eta 0:00:00
Paste your Google API key: ··········
🤖 Starting BaristaBot...
BaristaBot: Welcome to the BaristaBot café! Type `q` to quit. How may I serve you?
You: Coffee
BaristaBot: Okay, coffee. What size and type of coffee would you like?  We have espresso, Americano, cappuccino, and latte.  And sizes are small, medium, and large.

You: expresso
BaristaBot: Okay, espresso.  Any modifiers?  We have milk, sugar, and syrups (vanilla, hazelnut, caramel).

You: black
BaristaBot: Okay, one black espresso. Anything else for you?



You: 